## 곱셈 & 덧셈 계층 구현


In [ ]:
# Multiple
class MulLayer:
  def __init__(self):
    self.x = None
    self.y = None
  def forward(self, x, y):
    self.x = x
    self.y = y
    return x * y
  def backward(self, out):
    dx = out * self.y
    dy = out * self.x

    return dx, dy

In [ ]:
# Add
class AddLayer:
  def __init__(self):
    pass
  def forward(self, x, y):
    return x + y
  def backward(self, out):
    dx = out * 1
    dy = out * 1
    return dx, dy

## 활성화함수

In [ ]:
# ReLU
class Relu:
    def __init__(self):
      self.mask = None
    def forward(self, x):
      self.mask = (x <= 0)
      out = x.copy()
      out[self.mask] = 0
      return out
    def backward(self, out):
      out[self.mask] = 0
      x = out
      return x

# Sigmoid
class Sigmoid:
  def __init__(self):
    self.out = None
  def forward(self, x):
    out = 1 / (1+ np.exp(-x))
    self.out = out
    return out
  def backward(self, out):
    x = out * (1.0 - self.out) * self.out
    return x

## Affine & Softmax

In [ ]:
# Affine
class Affine:
  def __init__(self, W, b):
    self.W = W
    self.b = b
    self.x = None
    self.dW = None
    self.db = None
  
  def forward(self, x):
    self.x = x
    out = np.dot(x, self.W) + self.b
    return out

  def backward(self, out):
    dx = np.dot(out, self.W.T)
    self.dW = np.dot(self.x.T, out)
    self.db = np.sum(out, axis=0)
    return dx

# Softmax
class SoftmaxWithLoss:
  def __init__(self):
    self.loss = None
    self.y = None
    self.t = None

  def forward(self,x,t):
    self.t = t
    self.y = softmax(x)
    self.loss = cross_entropy_error(self.y, self,t)
    return self.loss

  def backward(self, dout=1):
    batch_size = self.t.shape[0]
    dx = (self.y - self.t) / batch_size
    return dx